In [ ]:
import logging

# Polyaxon
from polyaxon_client.tracking import Experiment

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
logger = logging.getLogger()


def model(log_learning_rate, max_depth=3, num_rounds=10, min_child_weight=5):
    model = XGBClassifier(
        learning_rate=10 ** log_learning_rate,
        max_depth=max_depth,
        num_rounds=num_rounds,
        min_child_weight=min_child_weight,
        objective='binary:logistic',
    )
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(pred, y_test)

In [ ]:
log_learning_rate_param=-3
max_depth_param=3
num_rounds_param=10
min_child_weight_param=5

In [ ]:
# Polyaxon
experiment = Experiment('iris')
experiment.create(framework='xgboost', tags=['examples'])
experiment.log_params(log_learning_rate=log_learning_rate_param,
                      max_depth=max_depth_param,
                      num_rounds=num_rounds_param,
                      min_child_weight=min_child_weight_param)

iris = load_iris()
X = iris.data
Y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Polyaxon
experiment.log_data_ref(data=X_train, data_name='x_train')
experiment.log_data_ref(data=y_train, data_name='y_train')
experiment.log_data_ref(data=X_test, data_name='X_test')
experiment.log_data_ref(data=y_test, data_name='y_train')

logger.info('Train model...')
accuracy = model(log_learning_rate=log_learning_rate_param,
                    max_depth=max_depth_param,
                    num_rounds=num_rounds_param,
                    min_child_weight=min_child_weight_param)
experiment.log_metrics(accuracy=accuracy)